# Sound Object Playback

The purpose of this script is to break out some 5-second sound objects from the Metatone touch archive and play them back on a device.

In [31]:
#%matplotlib inline
from __future__ import print_function
import os
import time
from datetime import timedelta
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
import seaborn as sns
import random
from touch_performance_player import TouchPerformancePlayer
import OSC


def clean_sound_object(frame):
    """Cleans up sound object frames by removing unneeded 
        columns and changing times to differences."""
    first_time = frame.index[0].to_pydatetime()
    output = frame[['x_pos','y_pos','velocity']]
    output['time'] = output.index
    output.time = (output.time - first_time).apply(timedelta.total_seconds)
    # output.time = output.time.diff() # adding 
    output.time = output.time.fillna(0)
    output = output.rename(columns={'x_pos': 'x', 'y_pos': 'y', 'velocity': 'velocity', 'time': 'time'})
    return output

def five_second_frame(row, frame):
    """
    Returns a dataframe of five seconds length before "time".
    """
    end_time = row.name
    frame = frame.between_time((end_time + timedelta(seconds=-5)).time(), end_time.time())
    if frame.empty:
        return np.nan
    return frame

### Load up data

- Loads metatone logs.
- Checks if output directory exists, otherwise creates it.

In [2]:
log_files = []
performances = []

output_directory = "individual-perfs-metatone-format/"
output_fileending = "-indperf.csv"

if not os.path.exists(output_directory):
    print("Creating Individual Perf Directory")
    os.makedirs(output_directory)

for local_file in os.listdir("data"):
    if local_file.endswith("-touches.csv"):
        log_files.append("data/" + local_file)
# for log in log_files:
    # do some stuff
    
log_frames = []
print("Loading all the frames.")
for log in log_files:
    log_frames.append(pd.DataFrame.from_csv(log,parse_dates=True,header=0))
print("Done Loading", len(log_frames), "logs.")

Loading all the frames.
Done Loading 163 logs.


In [48]:
import OSC
import random
from threading import Timer

PLAYBACK_TOUCH_PATTERN = "/metatone/playback/touch"
PLAYBACK_GESTURE_PATTERN = "/metatone/playback/gesture"

class TouchPerformancePlayer:

	def __init__(self):
		self.client = OSC.OSCClient()
		self.performers = {}
		self.local_address = ("localhost",5000)
		self.addPerformer("local","localhost",5000)

	def addPerformer(self, name, address, port):
		"""Adds a performer's address and port to the list"""
		self.performers[name] = (address, port)

	def sendTouch(self, performer, x, y, velocity):
	    """Sends an OSC message to trigger a touch sound."""
	    address = self.performers[performer]
	    self.client.sendto(OSC.OSCMessage(PLAYBACK_TOUCH_PATTERN, [performer,x, y, velocity]), address)

	def playPerformance(self, perf, performer = "local"):
		"""Schedule performance of a tiny performance dataframe."""
		for row in perf.iterrows():
			Timer(row[1]['time'],self.sendTouch,args=[performer,row[1]['x'],row[1]['y'],row[1]['velocity']]).start()


In [50]:
player = TouchPerformancePlayer()
player.addPerformer('charles','192.168.0.36',51200)

log = log_frames[120]
print(log.device_id.unique())
log_frame = log[log.device_id == '9A5116EA-2793-4C75-AC93-524C9EF550FD']
perf = clean_sound_object(log_frame)[:1000]
#print(log)

## Sound object processing routine.
# log_frame has to be a one-performer frame.
times = pd.DataFrame(log_frame.index,index=log_frame.index).resample('1s',how='count')
sound_objects = times.apply(five_second_frame, axis=1, frame=log_frame).dropna()

## Try one out:
player.playPerformance(clean_sound_object(sound_objects.ix[153]),'charles')

#player.playPerformance(perf,'charles')


#player.sendTouch('charles',283.5,497.5,5.0)



['24C41EC3-7ECC-40DA-B942-7ABE23017E74'
 '9A5116EA-2793-4C75-AC93-524C9EF550FD'
 'BD49B35A-8999-4987-9759-8D3F28D1292B'
 '1A56F5DD-C5B6-4E56-8690-E5A12BAA7E78']


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()


                                                       device_id  x_pos  \
time                                                                      
2015-04-29 18:11:57.393429  9A5116EA-2793-4C75-AC93-524C9EF550FD  518.5   
2015-04-29 18:11:57.943861  9A5116EA-2793-4C75-AC93-524C9EF550FD  606.0   
2015-04-29 18:11:58.645162  9A5116EA-2793-4C75-AC93-524C9EF550FD  716.5   
2015-04-29 18:11:59.285835  9A5116EA-2793-4C75-AC93-524C9EF550FD  796.5   
2015-04-29 18:11:59.310809  9A5116EA-2793-4C75-AC93-524C9EF550FD  793.5   
2015-04-29 18:11:59.311591  9A5116EA-2793-4C75-AC93-524C9EF550FD  792.5   
2015-04-29 18:11:59.346351  9A5116EA-2793-4C75-AC93-524C9EF550FD  885.0   
2015-04-29 18:11:59.663818  9A5116EA-2793-4C75-AC93-524C9EF550FD  945.0   
2015-04-29 18:12:00.209270  9A5116EA-2793-4C75-AC93-524C9EF550FD  964.5   
2015-04-29 18:12:00.274555  9A5116EA-2793-4C75-AC93-524C9EF550FD  991.5   

                            y_pos  velocity  
time                                         
2015-04

Exception in thread Thread-9118:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/threading.py", line 1073, in run
    self.function(*self.args, **self.kwargs)
  File "<ipython-input-48-5fc1283843cb>", line 22, in sendTouch
    address = self.performers[performer]
KeyError: 'charles'

Exception in thread Thread-9119:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/threading.py", line 1073, in run
    self.function(*self.args, **self.kwargs)
  File "<ipython-input-48-5fc1283843cb>", line 22, in sendTouch
    address = se

# Process touch logs



In [51]:
total_touches = 0
total_performances = 0
total_performers = 0
all_perfs_objects = []

print("Dividing all performances by player and converting to perf object format")

for log in log_frames:
    total_performances += 1
    for n in log.device_id.unique():
        total_performers += 1
        l = log[log.device_id == n]
        individual_log_title = l.index[0].to_pydatetime().strftime("%Y-%m-%d-%H-%M-%S")
        individual_log_title += "-" + n
        l = clean_sound_object(l)
        l = l.set_index('time')
        total_touches += l.x.count() # Add to total touches processed
        l.to_csv(output_directory + individual_log_title + output_fileending)
        all_individual_tiny_perfs.append(l)
                
print()
print("Processed", total_performances, "performances.")
print("There were", total_performers, "performers in total.")
print("Total touches recorded was:", total_touches)

print("Now saving a big file with all performances concatenated.")
all_individual_tiny_perfs = pd.concat(all_individual_tiny_perfs)
all_individual_tiny_perfs.to_csv("metatone_corpus_touches.csv")
print("Done.")



Dividing all performances by player and converting to tiny performance format.

Processed 163 performances.
There were 548 performers in total.
Total touches recorded was: 4298418
Now saving a big file with all performances concatenated.
Done.


### Unused Functions and Ideas

In [52]:


def split_and_classify_metatone_log(log_frame):
    return [],[]


#for performer in log_frame.device_id.unique():
#    sound_object_frames[performer] = times.apply(five_second_frame, axis=1, frame=log_frame, name=performer)

#out = times.apply(five_second_frame, axis=1, frame=log_frame, name='charles')

# 1. Split via performer
# 2. Split into rolling five second segments (should be a list)
# 3. Produce a feature vector and classify gesture for each segment (record separately)
# 4. Calculate time and moving columns
# 5. Save each non-empty segment into a CSV file.